## Business problem

Port terminals constantly strive to improve the efficiency of their operations through a careful
management of their berth facilities, machinery and personnel. 



The most important variables when planning terminal operations is knowing which vessels will arrive at the terminal and when.


MarineTraffic aims at being the best visibility providers by providing up to date vessel tracking data (using AIS), as well as additional derived information such as the estimated time of arrival (ETA) of a vessel to a port of interest.


AIS messages contain information on the port that the vessel is traveling to as well as the estimated time of arrival.

However, since ports may consist of more than one terminal, the exact terminal that the
vessel will visit is not known in advance making it difficult for MarineTraffic to assign future arrivals to terminals which, in turn, limits the ability to measure terminal congestion and calculate more accurate terminal arrival times.



A model which predicts the terminal a vessel will travel to has the potential to help all parties involved in a port call to plan their operations more effectively

# Data Description

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of **Hamburg and Port of Los Angeles**. The dataset contains the following fields

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of Hamburg and Port of Los Angeles. The dataset contains the following fields;


* **last_port**: Port where the *last* terminal call by vessel is recorded.



* **last_terminal**: The immediately previous *terminal* call of the vessel.


* **last_terminal_doc_timestamp**: Timestamp of previous terminal call.


* **current_port**: Port where the current terminal call by vessel is recorded.


* **current_terminal**: Current terminal call of the vessel.


* **shipname**: Name of the vessel.


* **dock_timestamp**: Timestamp of current terminal call.


* **GRT**: Vessel capacity (gross tonnage unit).


* **TEU**: Vessel capacity (twenty-foot equivalent Unit).


* **length**: Vessel length.


* **width**: Vessel width.

# Goals and Deliverable


The goal of this task is to implement & evaluate the accuracy of a solution that predicts the terminal
that a vessel will call.
**The product team claims that the history of terminals visited by a vessel in the past is a critical factor
that should be incorporated into the model**.

Some important steps that your solution would be expected to address and describe are the following:
* What features have you finally selected and engineered for your modeling approach? What led you to these choices? Why & how have you processed them?
* Which features seem to be the most important & how did you evaluate their importance?
* Do your findings agree with the product team’s insights discussed above? Before developing a ML model, how would you evaluate the importance/predictive power of one of the productidentified features as an independent variable?
* What type of prediction/training model have you chosen and why?
* How well does your predictive solution perform in terms of predicting the terminal a vessel will call?
* What different metrics/graphs can you use in order to understand when & why the algorithm fails/succeeds?
* What would be your baseline (i.e. a “naive” approach) to compare against?

# Solution Procedure

### problem understanding & EDA

In [1]:
# In order to understand the problem we have to dive into the data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler

In [3]:
raw_data = pd.read_csv("data/mt_terminal_calls.csv")

In [4]:
raw_data.head(5)

,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
0,BREMERHAVEN,North Sea Terminal,2020-01-02 16:12:00.000,HAMBURG,Eurogate Container Terminal Hamburg,HEINRICH EHLER,2020-01-03 06:20:00.000,17488,1421,168.11,26.80
1,TILBURY,London Container Terminal,2020-01-01 21:56:00.000,HAMBURG,C. Steinweg Multipurpose Terminal,HENNEKE RAMBOW,2020-01-03 07:45:00.000,9981,868,134.40,22.74
2,ROTTERDAM MAASVLAKTE,Rotterdam World Gateway Terminal,2020-01-02 12:06:00.000,HAMBURG,HHLA Container Terminal Burchardkai,CMA CGM TANGER,2020-01-03 20:20:00.000,9966,1118,147.80,23.28
3,ROTTERDAM WAALHAVEN,RST Waalhaven,2020-01-01 16:05:00.000,HAMBURG,HHLA Container Terminal Burchardkai,NIEVES B,2020-01-03 20:28:00.000,10318,1036,151.72,23.40
4,GDYNIA,Gdynia Container Terminal,2020-01-01 14:45:00.000,HAMBURG,HHLA Container Terminal Burchardkai,JUDITH,2020-01-04 05:05:00.000,16023,1440,170.02,25.19


In order to make sense of the markets team claim of the historical importance of each ship,
is to check if any of those ships have actually performed a "full circle" from HAMBURG/LA to HAMBURG/LA

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12054 entries, 0 to 12053
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   last_port                    12054 non-null  object 
 1   last_terminal                12054 non-null  object 
 2   last_terminal_doc_timestamp  12054 non-null  object 
 3   current_port                 12054 non-null  object 
 4   current_terminal             12054 non-null  object 
 5   shipname                     12054 non-null  object 
 6   doc_timestamp                12054 non-null  object 
 7   grt                          12054 non-null  int64  
 8   teu                          12054 non-null  int64  
 9   length                       12054 non-null  float64
 10  width                        12054 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.0+ MB


In [6]:
last_ports = raw_data.loc[:, "last_port"].unique().tolist()

In [7]:
current_port = raw_data.loc[:, "current_port"].unique().tolist()

In [8]:
print(f"{len(current_port)}")

2


In [9]:
print(f"{len(last_ports)}")

165


In [10]:
set(current_port).intersection(last_ports)

set()

the empty set here signifies that there are no roundtrips of the ships.

### lets investigate the timeline of some ships

In [11]:
print(f"the unique ships in this dataset are {len(raw_data.shipname.unique())}")

the unique ships in this dataset are 1397


In [12]:
ships_gone_both = []

In [13]:
for ship in raw_data.shipname.unique():
    dest = len(raw_data.loc[raw_data["shipname"]==ship,"current_port"].unique())
    if dest !=1:
        ships_gone_both.append(ship)

In [14]:
print(f"{len(ships_gone_both)} have gone to both ports.")

101 have gone to both ports.


In [15]:
user_defined = 5
for ship_no,(name,group) in enumerate(raw_data.groupby("shipname")):
    print(f"ship: {name}")
    display(group.sort_values(by="last_terminal_doc_timestamp"))
    if ship_no > user_defined -1:
        break

ship: A DAISEN


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
7497,SHANGHAI,Shanghai Mingdong Container Terminal,2021-09-08 06:13:00.000,LOS ANGELES,Everport Services Terminal,A DAISEN,2021-10-07 11:55:00.000,18326,1740,175.46,27.69
9058,QINGDAO,QQCT,2022-01-12 07:56:00.000,LOS ANGELES,TraPac Terminal,A DAISEN,2022-02-17 11:00:00.000,18326,1740,175.46,27.69
9666,NINGBO,Beilun International Container Terminals,2022-03-22 14:58:00.000,LOS ANGELES,TraPac Terminal,A DAISEN,2022-04-12 10:23:00.000,18326,1740,175.46,27.69
10272,NINGBO,Beilun International Container Terminals,2022-05-13 15:25:00.000,LOS ANGELES,Yusen Terminal Inc.,A DAISEN,2022-06-03 22:19:00.000,18326,1740,175.46,27.69


ship: A LA MARINE


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
122,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-01-06 15:48:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-01-14 22:49:00.000,16023,1440,170.06,25.2
287,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-01-22 23:14:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-01-28 23:34:00.000,16023,1440,170.06,25.2
446,ST PETERSBURG,Bulk cargo Quay,2020-02-05 22:01:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-02-12 01:00:00.000,16023,1440,170.06,25.2
586,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-02-20 20:52:00.000,HAMBURG,Cruise Center Steinwerder,A LA MARINE,2020-02-25 11:22:00.000,16023,1440,170.06,25.2
771,ST PETERSBURG,Container Terminal Saint-Petersburg,2020-03-06 22:48:00.000,HAMBURG,HHLA Container Terminal Burchardkai,A LA MARINE,2020-03-12 10:42:00.000,16023,1440,170.06,25.2
917,BREMERHAVEN,EUROGATE CTB,2020-03-24 07:17:00.000,HAMBURG,HHLA Container Terminal Tollerort,A LA MARINE,2020-03-25 13:31:00.000,16023,1440,170.06,25.2
1034,BREMERHAVEN,North Sea Terminal,2020-04-05 02:03:00.000,HAMBURG,HHLA Container Terminal Burchardkai,A LA MARINE,2020-04-05 19:26:00.000,16023,1440,170.06,25.2
1085,BREMERHAVEN,EUROGATE CTB,2020-04-08 21:15:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-04-09 12:05:00.000,16023,1440,170.06,25.2
1182,COPENHAGEN,CMP CONTAINER TERMINAL,2020-04-17 06:45:00.000,HAMBURG,HHLA Container Terminal Altenwerder,A LA MARINE,2020-04-18 23:34:00.000,16023,1440,170.06,25.2
1264,BREMERHAVEN,EUROGATE CTB,2020-04-20 20:50:00.000,HAMBURG,Eurogate Container Terminal Hamburg,A LA MARINE,2020-04-25 03:51:00.000,16023,1440,170.06,25.2


ship: A.IDEFIX


...

,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
878,ANTWERP,PSA Europa Terminal,2020-03-20 00:37:00.000,HAMBURG,HHLA Container Terminal Altenwerder,ADAMS,2020-03-22 12:14:00.000,66462,5928,279.7,40.0
1430,ANTWERP,PSA Europa Terminal,2020-05-08 00:39:00.000,HAMBURG,HHLA Container Terminal Altenwerder,ADAMS,2020-05-11 02:02:00.000,66462,5928,279.7,40.0
1973,ANTWERP,PSA Europa Terminal,2020-06-26 23:00:00.000,HAMBURG,HHLA Container Terminal Altenwerder,ADAMS,2020-06-29 12:16:00.000,66462,5928,279.7,40.0


ship: ADELINA D


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
7354,BREMERHAVEN,MSC Gate,2021-09-24 11:11:00.000,HAMBURG,Eurogate Container Terminal Hamburg,ADELINA D,2021-09-25 05:21:00.000,15487,1578,168.0,25.3
7479,BREMERHAVEN,North Sea Terminal,2021-10-05 04:45:00.000,HAMBURG,Eurogate Container Terminal Hamburg,ADELINA D,2021-10-05 22:17:00.000,15487,1578,168.0,25.3
7627,BREMERHAVEN,North Sea Terminal,2021-10-16 23:36:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2021-10-17 16:42:00.000,15487,1578,168.0,25.3
7743,NORVIK,Stockholm Norvik Container Terminal,2021-10-25 05:24:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2021-10-27 12:19:00.000,15487,1578,168.0,25.3
7886,BREMERHAVEN,North Sea Terminal,2021-11-06 07:23:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2021-11-07 09:47:00.000,15487,1578,168.0,25.3
8000,GAVLE,Yilport Container Terminal,2021-11-12 12:34:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2021-11-17 14:04:00.000,15487,1578,168.0,25.3
8138,BREMERHAVEN,EUROGATE CTB,2021-11-28 11:28:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2021-11-29 04:07:00.000,15487,1578,168.0,25.3
8312,BREMERHAVEN,EUROGATE CTB,2021-12-13 00:14:00.000,HAMBURG,HHLA Container Terminal Tollerort,ADELINA D,2021-12-14 06:17:00.000,15487,1578,168.0,25.3
8430,BREMERHAVEN,EUROGATE CTB,2021-12-22 21:06:00.000,HAMBURG,Eurogate Container Terminal Hamburg,ADELINA D,2021-12-23 22:16:00.000,15487,1578,168.0,25.3
8554,GAVLE,Yilport Container Terminal,2021-12-31 13:16:00.000,HAMBURG,HHLA Container Terminal Burchardkai,ADELINA D,2022-01-06 01:44:00.000,15487,1578,168.0,25.3


ship: ADILIA I


,last_port,last_terminal,last_terminal_doc_timestamp,current_port,current_terminal,shipname,doc_timestamp,grt,teu,length,width
3217,ROTTERDAM MAASVLAKTE,Hutchison Ports Delta II,2020-10-07 20:12:00.000,HAMBURG,HHLA Container Terminal Altenwerder,ADILIA I,2020-10-09 04:33:00.000,9701,830,140.48,22.84


In [16]:
terminal_ham = raw_data.loc[raw_data["current_port"]=="HAMBURG", "current_terminal"].unique().tolist()

In [17]:
terminal_la = raw_data.loc[raw_data["current_port"]=="LOS ANGELES", "current_terminal"].unique().tolist()

In [18]:
# Essentially these here are our labels.

In [19]:
set(terminal_ham).intersection(set(terminal_la)),set(terminal_la).intersection(set(terminal_ham))

(set(), set())

In [20]:
print("possible final terminals", len(terminal_ham)+len(terminal_la))

possible final terminals 19


In [21]:
raw_data.columns

Index(['last_port', 'last_terminal', 'last_terminal_doc_timestamp',
       'current_port', 'current_terminal', 'shipname', 'doc_timestamp', 'grt',
       'teu', 'length', 'width'],
      dtype='object')

In [22]:
raw_data["port_terminal"] = raw_data.apply(lambda x: f"{x['current_port']}_{x['current_terminal']}",axis=1)

In [23]:
print(raw_data["port_terminal"].unique().shape[0])

19


In [24]:
raw_data.apply(lambda x: f"{x['last_port']}_{x['last_terminal']}",axis=1).unique().shape

(269,)

In [25]:
# if we keep in mind the port and the last terminal in mind we will have a severe case of sparse matrix
# since we're going to have to create one hot representation for that feature.

In [26]:
# first attempt

In [27]:
raw_data.columns

Index(['last_port', 'last_terminal', 'last_terminal_doc_timestamp',
       'current_port', 'current_terminal', 'shipname', 'doc_timestamp', 'grt',
       'teu', 'length', 'width', 'port_terminal'],
      dtype='object')

In [28]:
kept_cols =['last_terminal_doc_timestamp', 'current_port', 'current_terminal', 'shipname', 'doc_timestamp', 'grt','teu', 'length', 'width', 'port_terminal']

In [29]:
# distance could potentially be a much greater feature than everything else?

In [30]:
kept_cols.append("diff")

In [31]:
raw_data["diff"] = (pd.to_datetime(raw_data.loc[:, "doc_timestamp"]) - pd.to_datetime(raw_data.loc[:, "last_terminal_doc_timestamp"]))

In [32]:
raw_data = raw_data.loc[:, kept_cols]

In [33]:
raw_data = raw_data.loc[:, ["grt","teu","length", "width", "diff", "port_terminal"]]

In [34]:
raw_data.describe()

,grt,teu,length,width,diff
count,12054.000000,12054.000000,12054.000000,12054.000000,12054
mean,60620.134063,5541.533018,244.204202,34.706638,6 days 17:26:49.094076655
std,57664.198201,5446.331962,90.141575,11.647131,9 days 00:47:17.448698429
min,3992.000000,508.000000,99.950000,17.900000,0 days 09:39:00
25%,11662.000000,1036.000000,154.520000,23.530000,1 days 21:54:15
50%,40146.000000,4045.000000,259.000000,32.200000,2 days 22:20:30
75%,95680.000000,8566.000000,333.000000,45.600000,6 days 22:08:15
max,236583.000000,24004.000000,400.000000,62.000000,197 days 21:37:00


In [35]:
raw_data["diff"] = raw_data.apply(lambda x: x['diff'].seconds,axis=1)

In [36]:
raw_data

,grt,teu,length,width,diff,port_terminal
0,17488,1421,168.11,26.80,50880,HAMBURG_Eurogate Container Terminal Hamburg
1,9981,868,134.40,22.74,35340,HAMBURG_C. Steinweg Multipurpose Terminal
2,9966,1118,147.80,23.28,29640,HAMBURG_HHLA Container Terminal Burchardkai
3,10318,1036,151.72,23.40,15780,HAMBURG_HHLA Container Terminal Burchardkai
4,16023,1440,170.02,25.19,51600,HAMBURG_HHLA Container Terminal Burchardkai
...,...,...,...,...,...,...
12049,39753,3100,228.15,32.25,71760,HAMBURG_Eurogate Container Terminal Hamburg
12050,35835,3078,223.30,32.20,15720,HAMBURG_Eurogate Container Terminal Hamburg
12051,17982,1380,169.95,26.90,59400,HAMBURG_HHLA Container Terminal Altenwerder
12052,10585,1000,151.72,23.40,69300,HAMBURG_Eurogate Container Terminal Hamburg


# first simple baseline

In [37]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

encoder.fit(raw_data.loc[:,["port_terminal"]].values.reshape(-1,1))



OneHotEncoder()

In [38]:
# since we actually do know the destination port, and those two ports do have different
# number of

In [39]:
model = LogisticRegression()

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(raw_data.loc[:, raw_data.columns!="port_terminal"], raw_data.loc[:,["port_terminal"]], test_size=0.2, random_state=42)

In [42]:
y_train = encoder.transform(y_train)
y_train = np.argmax(y_train, axis=1)
y_train = pd.DataFrame(y_train,columns=["label"])
y_test = encoder.transform(y_test)
y_test = np.argmax(y_test, axis=1)
y_test = pd.DataFrame(y_test,columns=["label"])

C:\Users\dimitrisl\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\dimitrisl\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [43]:
model.fit(X_train,y_train)

C:\Users\dimitrisl\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dimitrisl\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [44]:
y_pred = model.predict(X_test)

In [46]:
print(f1_score(y_test, y_pred, average="micro"))

0.24678556615512234


In [47]:
print(f1_score(y_test, y_pred, average="macro"))

0.05239921087671952


In [49]:
accuracy_score(y_test, y_pred)

0.24678556615512234

In [50]:
model = SVC()
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))
print(f1_score(y_test, y_pred, average="macro"))

0.26627955205309
0.04951302199284246


In [51]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

In [52]:
model = RandomForestClassifier()

In [53]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

C:\Users\dimitrisl\AppData\Local\Temp\ipykernel_6412\1897702526.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y_train)


In [54]:
print(f1_score(y_test, y_pred, average="micro"))

0.5868934052260473


In [55]:
print(f1_score(y_test, y_pred, average="macro"))

0.45082557559024167


In [56]:
train_scaler = RobustScaler()

In [57]:
X_train = train_scaler.fit_transform(X_train)

In [58]:
X_train

array([[ 1.26787988,  1.32297726,  0.5884822 ,  0.85183507,  0.6417704 ],
       [-0.35918066, -0.4208091 , -0.69711216, -0.42863616, -0.47994467],
       [-0.35918066, -0.4208091 , -0.69711216, -0.42863616,  0.3582296 ],
       ...,
       [ 1.85039098,  1.95068747,  0.78912021,  1.19619393, -0.11341632],
       [-0.26993894, -0.3469064 , -0.50867473, -0.24467603,  0.27109267],
       [ 0.08439758,  0.02022739,  0.        ,  0.23108292,  0.04979253]])

In [59]:
eval_scaler = RobustScaler()

In [60]:
X_test = eval_scaler.fit_transform(X_test)

In [61]:
model = RandomForestClassifier()

In [62]:
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

In [63]:
print(f1_score(y_test, y_pred, average="micro"))

0.33513065118208213


In [64]:
print(f1_score(y_test, y_pred, average="macro"))

0.20473561555493963


In [65]:
model = LogisticRegression()
model.fit(X_train, y_train.values.ravel())

y_pred = model.predict(X_test)

print(f1_score(y_test, y_pred, average="micro"))
print(f1_score(y_test, y_pred, average="macro"))

0.2625466611364579
0.08834413050441701


C:\Users\dimitrisl\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Enstablish simple baselines

### Can we do better?

# Possible future work

* Given the accuracy of your model, what would be the next steps in terms of deploying a solution such that the model’s value to the business is maximised?